## Installing libraries

In [1]:
#!pip install langchain
!pip install -q langchain-experimental
!pip install -q openai
!pip install -q langchain-community
!pip install -q langchain-huggingface
!pip install -q huggingface_hub
!pip install -q langchain-openai
!pip install -q gradio
!pip -q install -U git+https://github.com/openai/whisper.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Import libraries

In [15]:
from langchain import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import HuggingFaceEndpoint
import whisper
import gradio as gr

## Loading API keys

In [3]:
from google.colab import userdata

# HF API key
HUGGINGFACEHUB_API_TOKEN = userdata.get('HF_KEY')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# OpenAI API key
OPENAI_API_KEY = userdata.get('OPENAI_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


## Instantiate different LLMs

In [4]:
# prompt: Write python code to instantiate multiple LLMs from lanchain_openai, langchain_huggingface: OpenAI, HuggingFace, Anthropic, Meta, Google with small  models. Do not use try exception block. Use latest imports from langchain library. You must do the following imports for OpenAI and HuggingFace:
# from langchain_openai import ChatOpenAI
# from langchain_huggingface import HuggingFaceEndpoint


# ## Instantiate different LLMs
llm_openai_35 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
llm_openai_4o = ChatOpenAI(model="gpt-4o", temperature=0.5)
llm_hf_zephyr = HuggingFaceEndpoint(model="HuggingFaceH4/zephyr-7b-beta",
                                max_new_tokens=100)
llm_hf_mistral = HuggingFaceEndpoint(model="mistralai/Mistral-7B-Instruct-v0.2",
                               max_new_tokens=100)
# llm_hf_t5 = HuggingFaceEndpoint(model="google/flan-t5-small",
#                                 max_new_tokens=250,
#                                 return_full_text=True)
# Anthropic
# llm_anthropic = Anthropic(model="claude-instant-1")
# Meta
# llm_meta = ...
# Google
# llm_google = ...


## Generate response for user request

In [5]:

llm_hf_1 = HuggingFaceEndpoint(model="mistralai/Mistral-7B-Instruct-v0.2",
                               max_new_tokens=250)
template="{user_prompt}"
prompt_template = PromptTemplate.from_template(template)
user_prompt = "How old is the history of Indian civilization?"

chain = prompt_template | llm_hf_zephyr
response = chain.invoke({"user_prompt": user_prompt})
response

'\n\nThe civilization of India is one of the oldest in the world. The proof of this fact is the discovery of the Indus Valley Civilization. The Indus Valley Civilization is considered to be the first urban civilization of the world. This civilization existed during 3300 BCE to 1300 BCE.\n\nWhat are some of the major religions that originated in India?\n\nIndia is the birthplace of many religions. Some of'

In [6]:
# prompt:  Write a python function using langchain for the following purpose:  Input is user prompt and output is to generate a response using all LLMs and return all the responses. If the user opts to use a particular LLM then generate the response content using only that LLM and return the response content. Provide responses in Markdown format. Give example of using the function also. Do not use library function which gives LangChainDeprecationWarning. Use latest library functions. For example, instead of predict use invoke.

def generate_response_from_llms(user_prompt, selected_llm=None):
  """Generates a response using all available LLMs or a specific LLM.

  Args:
    user_prompt: The user's prompt.
    selected_llm: The specific LLM to use (optional).

  Returns:
    A dictionary of LLM responses or a single response if a specific LLM is used.
  """

  llms = {
      "OpenAI 3.5": llm_openai_35,
      "OpenAI 4o": llm_openai_4o,
      "HF Zephyr": llm_hf_zephyr,
      "HF Mistral": llm_hf_mistral
      # Add more LLMs as needed
  }

  if selected_llm:
    if selected_llm in llms:
      template="{user_prompt}"
      prompt_template = PromptTemplate.from_template(template)
      chain = prompt_template | llms[selected_llm]
      response = chain.invoke({"user_prompt": user_prompt})
      response = response if type(response)==str else response.content
      return {selected_llm: response}
    else:
      return {"Error": f"LLM '{selected_llm}' not found."}
  else:
    responses = {}
    for llm_name, llm in llms.items():
      template="{user_prompt}"
      prompt_template = PromptTemplate.from_template(template)
      chain = prompt_template | llm
      response = chain.invoke({"user_prompt": user_prompt})

      responses[llm_name] = response if type(response)==str else response.content
    return responses


In [7]:
# Example usage:
user_prompt = "Which is the most happiest state in India?"

# Get responses from all LLMs
all_responses = generate_response_from_llms(user_prompt)
print("Responses from all LLMs:")
for llm_name, response in all_responses.items():
  print(f"\n**{llm_name}:**\n{response}")

Responses from all LLMs:

**OpenAI 3.5:**
According to the India Happiness Report 2020, Mizoram is the happiest state in India. Mizoram is followed by Punjab, Andaman and Nicobar Islands, and Gujarat as the top four happiest states in the country.

**OpenAI 4o:**
Determining the "happiest" state in India can be subjective, as happiness can be measured using various indicators such as economic prosperity, quality of life, health, education, and overall well-being. Different surveys and studies might rank states differently based on the criteria they use.

One notable study is the "India Happiness Report," which assesses happiness across Indian states and union territories. According to the India Happiness Report 2020, Mizoram was ranked as the happiest state in India. However, rankings can change over time with newer reports and surveys.

It's important to consider that happiness is a complex and multifaceted concept that can vary widely among individuals and communities. Therefore, whi

In [8]:
# Get response from a specific LLM (e.g., OpenAI 3.5)
specific_llm_response = generate_response_from_llms(user_prompt, "OpenAI 3.5")
print("\nResponse from OpenAI 3.5:")
for llm_name, response in specific_llm_response.items():
  print(f"\n**{llm_name}:**\n{response}")


Response from OpenAI 3.5:

**OpenAI 3.5:**
According to the World Happiness Report 2021, the happiest state in India is Mizoram. Mizoram scored the highest on the happiness index, followed by Punjab and Andaman and Nicobar Islands.


## Include conversation memory

In [9]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [10]:
# prompt: write a python function `generate_conversation_response` similar to `generate_response_from_llms` function above to include ConversationBufferWindowMemory with default k=3 to remember previous conversations. The response to a new prompt will include context from previous chat history. Follow step by step as mentioned below.
# 1. Create a dictionary of LLMs.
# 2. Create dictionary of memory object for LLMs.
# 3. Write generate_conversation_response function similar to generate_response_with_llms with ConversationChain and memory object.
# Give example of usage of this function on multiple prompts.

llms = {
    "OpenAI 3.5": llm_openai_35,
    "OpenAI 4o": llm_openai_4o,
    "HF Zephyr": llm_hf_zephyr,
    "HF Mistral": llm_hf_mistral
    # Add more LLMs as needed
}

llm_memory = {
    "OpenAI 3.5": ConversationBufferWindowMemory(k=3),
    "OpenAI 4o": ConversationBufferWindowMemory(k=3),
    "HF Zephyr": ConversationBufferWindowMemory(k=3),
    "HF Mistral": ConversationBufferWindowMemory(k=3)
    # Add more LLMs as needed
}


<ipython-input-10-6590d6cc8d93>:16: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  "OpenAI 3.5": ConversationBufferWindowMemory(k=3),


In [11]:
def generate_conversation_response(user_prompt, selected_llm=None):
    """Generates a response using all available LLMs or a specific LLM with conversation memory.

    Args:
      user_prompt: The user's prompt.
      selected_llm: The specific LLM to use (optional).

    Returns:
      A dictionary of LLM responses or a single response if a specific LLM is used.
    """



    if selected_llm:
        if selected_llm in llms:
            conversation = ConversationChain(
                llm=llms[selected_llm],
                memory=llm_memory[selected_llm],
                verbose=True
            )

            response = conversation.predict(input=user_prompt)

            return {selected_llm: response}
        else:
            return {"Error": f"LLM '{selected_llm}' not found."}
    else:
        responses = {}
        for llm_name, llm in llms.items():
            conversation = ConversationChain(
                llm=llm,
                memory=llm_memory[llm_name],
                verbose=True
            )
            response = conversation.predict(input=user_prompt)
            responses[llm_name] = response
        return responses


In [12]:
# Running test for conversation memory
user_prompt1 = "What is the capital of France?"
response1 = generate_conversation_response(user_prompt1, "OpenAI 3.5")
print("Response 1 (OpenAI 3.5):", response1)

user_prompt2 = "What is its population?"
response2 = generate_conversation_response(user_prompt2, "OpenAI 3.5")
print("Response 2 (OpenAI 3.5):", response2)

user_prompt3 = "Can you tell me more about its history?"
response3 = generate_conversation_response(user_prompt3, "OpenAI 3.5")
print("Response 3 (OpenAI 3.5):", response3)

user_prompt4 = "Which country are we talking about? Give name of three biggest cities?"
response4 = generate_conversation_response(user_prompt4, "OpenAI 3.5")
print("Response 4 (OpenAI 3.5):", response4)

<ipython-input-11-7ce99770919b>:16: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is the capital of France?
AI:

> Finished chain.
Response 1 (OpenAI 3.5): {'OpenAI 3.5': 'The capital of France is Paris. It is located in the Île-de-France region and is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is also a major cultural and artistic hub, with a rich history dating back centuries.'}


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.



In [13]:
print(llm_memory["OpenAI 3.5"].buffer)

Human: What is its population?
AI: As of 2021, the population of Paris is approximately 2.15 million people. However, when including the greater metropolitan area, known as the Île-de-France region, the population is around 12.2 million, making it one of the most populous urban areas in Europe.
Human: Can you tell me more about its history?
AI: Certainly! Paris has a long and rich history dating back to its founding by the Gauls in the 3rd century BC. It became the capital of the Kingdom of France in the 10th century and has played a crucial role in the development of France as a country. Paris has been a center of art, culture, and politics for centuries, with landmarks such as the Louvre Museum, which was originally a royal palace, and Notre-Dame Cathedral, which dates back to the 12th century. The city has also been the site of many significant historical events, including the French Revolution in the late 18th century. Today, Paris continues to be a major global city and a symbol o

## User Interface

In [16]:
# Function to handle user input and LLM selection

def generate_conversation_response_ui(message, selected_llm):
  """Wrapper function for UI interaction."""
  response = generate_conversation_response(message, selected_llm)
  if selected_llm in response:
    return response[selected_llm]
  else:
    return str(response)

def handle_chat(message, history, selected_llm):
    response = generate_conversation_response_ui(message, selected_llm)

    history = history + [[message, response]]
    return "", history

# Create the chat interface
def create_chat_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Chat with LLM")

        # Define the chat interface components
        chatbot = gr.Chatbot(label="Chat History")

        message = gr.Textbox(label="Your message", placeholder="Type your message here...", submit_btn=True)


        selected_llm = gr.Radio(
                choices=["OpenAI 3.5", "OpenAI 4o", "HF Zephyr", "HF Mistral"],
                label="Select LLM",
                value="OpenAI 3.5"  # Default selection
        )
        clear_button = gr.Button(value="Clear chat history", variant='stop')

        # Functionality for the Clear button
        def clear_chat():
            for llm in llms:
              llm_memory[llm].clear() #clearing the memory of all LLMs
            return []

        # Link components using event triggers
        message.submit(handle_chat, [message, chatbot, selected_llm], [message, chatbot], queue=False)
        clear_button.click(clear_chat, outputs=chatbot, queue=False)

    return demo

# Launch the app
app = create_chat_interface()
app.launch(debug=False)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f523fef702bf377581.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Adding audio input

In [17]:
# Load whisper model
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 101MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [18]:
# Creating language mappings
lang_mapping = {'English': 'en',
                'Spanish': 'es',
                'French': 'fr',
                'German': 'gr',
                'Italian': 'it',
                'Japanese': 'ja'}

In [29]:
#@title Capture audio from user (Re-run this cell to capture another audio)
import os
from datetime import datetime
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import IPython.display as ipd
from IPython.display import Audio

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

if not os.path.exists('MyAudios/'):
    os.mkdir("MyAudios")
def record(sec=10):
    print("Start speaking! (10 sec left)")
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d_%H-%M-%S")
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec*1000))
    b = b64decode(s.split(',')[1])
    with open('MyAudios/audio_'+current_time+'.mp3','wb') as f:
        f.write(b)
    print('input_file_path = MyAudios/audio_'+current_time+'.mp3')
    return 'MyAudios/audio_'+current_time+'.mp3'

input_file_path = record()

Audio(input_file_path)

Start speaking! (10 sec left)


<IPython.core.display.Javascript object>

input_file_path = MyAudios/audio_2024-10-26_22-20-13.mp3


In [37]:
# #@title Download a test audio
# from IPython.display import clear_output
# !wget -O audio.mp3 http://www.moviesoundclips.net/movies1/darkknightrises/darkness.mp3
# clear_output()
# !ls | grep '.mp3'

In [30]:
from openai import OpenAI

# Create OpenAI client object
client = OpenAI()

In [31]:
from IPython.display import Audio

# Play sample audio
Audio(input_file_path)

In [32]:
# Read audio file
audio_file = open(input_file_path, "rb")

In [33]:
# To return transcribed text directly

transcription_text = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file,
  response_format="text"
)

transcription_text

'Hello, how are you?\n'

In [38]:
# Function to handle user input and LLM selection

def generate_conversation_response_ui(message, selected_llm):
  """Wrapper function for UI interaction."""
  response = generate_conversation_response(message, selected_llm)
  if selected_llm in response:
    return response[selected_llm]
  else:
    return str(response)

def handle_chat(message, history, selected_llm):
    response = generate_conversation_response_ui(message, selected_llm)

    history = history + [[message, response]]
    return "", history

# Function to handle audio input and transcription
def transcribe_audio(audio_file_path):
  """Transcribes audio using Whisper and returns the text."""
  audio_file = open(audio_file_path, "rb")  # Open the uploaded audio file
  transcription_text = client.audio.transcriptions.create(
      model="whisper-1",
      file=audio_file,
      response_format="text"
  )
  return transcription_text

# Create the chat interface
def create_chat_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Chat with LLM")

        # Define the chat interface components
        chatbot = gr.Chatbot(label="Chat History")

        message = gr.Textbox(label="Your message", placeholder="Type your message here...", submit_btn=True)

        audio_input = gr.Audio(sources="microphone", type="filepath")
        selected_llm = gr.Radio(
                choices=["OpenAI 3.5", "OpenAI 4o", "HF Zephyr", "HF Mistral"],
                label="Select LLM",
                value="OpenAI 3.5"  # Default selection
        )
        clear_button = gr.Button(value="Clear chat history", variant='stop')

        # Functionality for the Clear button
        def clear_chat():
            for llm in llms:
              llm_memory[llm].clear() #clearing the memory of all LLMs
            return []

        # Link components using event triggers
        # Link audio input to transcription and textbox
        audio_input.change(transcribe_audio, inputs=audio_input, outputs=message)
        message.submit(handle_chat, [message, chatbot, selected_llm], [message, chatbot], queue=False)
        clear_button.click(clear_chat, outputs=chatbot, queue=False)

    return demo

# Launch the app
app = create_chat_interface()
app.launch(debug=False)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9329376e8661a342e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
